#### Import Modlues

In [1]:
import sys, os
import cohere
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join('../Scripts')))
#tg

In [18]:
#Import modules from script
from reader import ReadFile
from pipeline import Prompt
from preeprocessing import DataPreprocesser

#### Read Cleaned Data

In [19]:
#Initialize file and API_KEY
file = ReadFile()
propmt_pipeline = PromptPipeline()
API_KEY = 'mxiQHQDT91By3aDK7pjlghcJOm9qHfNZJ8a9Tyur'


In [36]:
#train_data =  file.dvc_get_data('../Data/relations_dev.json', 'vj')

In [28]:
test_data = reader.dvc_get_data('../Data/relations_test.json', 'vt')

##### Initialize our Cohere Client

In [6]:
co = cohere.Client(API_KEY)

use tuned models based on out extracted entities

In [38]:
my_vals = propmt_pipeline.extract_values(train_data)
with open("../data/relations_dev_fine_tune.txt", 'w')as f:
    f.write(my_vals)
#We Extract all the entities including job description text and we will feed this in to our model four fine tuning in the cohere website

In [39]:
my_vals = propmt_pipeline.extract_values(test_data)
with open("../data/relations_test_fine_tune.txt", 'w')as f:
    f.write(my_vals)

#### After training on cohere dashboard we will give it a try

In [40]:
val = propmt_pipeline.extract_values(train_data[:5])

In [94]:
prompt = val
prompt += f"\nJob Description: {test_data[2]['document'].strip()}"
prompt

"Job Description: Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience\nDIPLOMA: Bachelor\nDIPLOMA_MAJOR: Mechanical Engineering,Physical Science\nEXPERIENCE: 3+ years\nSKILLS: developing,fiber optic cables,connector related products\n\n--\nJob Description: 10+ years of software engineering work experience. Technical experience in release automation engineering, CI/

In [91]:
response = co.generate( 
model='large', 
prompt=prompt,
max_tokens=400, 
temperature=0.5, 
k=0, 
p=1, 
frequency_penalty=0, 
presence_penalty=0, 
stop_sequences=["--"], 
return_likelihoods='NONE') 
print('Prediction: {}'.format(response.generations[0].text))

Prediction: 
Experience in the use of large-scale infrastructure such as cluster, storage, networking, and/or compute.
Experience in statistical analysis of data from different sources.
Experience in developing and debugging in C/C++, Python, C# and/or Java.

DIPLOMA: Ph.D.
DIPLOMA_MAJOR: Computer Science
EXPERIENCE: 1+ year(s)
SKILLS: AI,computer science,statistics,applied mathematics,data science,technical research,public outreach

--


In [41]:
val = propmt_pipeline.extract_values(train_data[:5])

In [100]:
# prompt = val
prompt = f"Job Description: {test_data[2]['document'].strip()} --"
prompt

'Job Description: BA/BS\n5+ years of program or project management experience\n2+ years of technical project/program management experience\nTrack record of operating independently\nExperience understanding user needs, gathering requirements, and defining scope\nCommunication experience interacting with a variety of audiences from engineers, to vendors, to research leaders\nTrack record of building cross-functional relationships\n\nPREFERRED \nExperience working with UX Research and/or UX Design --'

In [101]:
response = co.generate( 
model='large', 
prompt=prompt,
max_tokens=400, 
temperature=0.5, 
k=0, 
p=1, 
frequency_penalty=0, 
presence_penalty=0, 
stop_sequences=["--"], 
return_likelihoods='NONE') 
print('Prediction: {}'.format(response.generations[0].text))
#Try an other fine tuned model, tuned with a larger model(medium), the previous one was small

Prediction:  especially in a consumer-facing product
Experience managing a team of UX Researchers and/or UX Designers
Experience managing a team of engineers
Experience managing a team of product managers
Experience working in a startup environment
Experience with Agile methodologies
Experience with user research tools and techniques

DIPLOMA 
5+ years of program or project management experience
2+ years of technical project/program management experience
Track record of operating independently
Experience understanding user needs, gathering requirements, and defining scope
Communication experience interacting with a variety of audiences from engineers, to vendors, to research leaders
Track record of building cross-functional relationships

PREFERRED 
Experience working with UX Research and/or UX Design --


the new fine tuned model is performing well. so we will modify our send_request_to_cohere to accept model name`